# Indice espectrales NDFI en GEE

(indice Normalizado de Vegetacion Fraccionada)

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

## Filtro zona estudio mediante GEE

In [2]:
# Agregar un shp GEE
zona_ee = ee.FeatureCollection("users/bravomoralesnino/SHP/ZONA_19S")

In [3]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='ESRI')

In [4]:
geometria = zona_ee.geometry()
Map.centerObject(geometria,11)

In [5]:
Map.addLayer(zona_ee, {"color" : "00FF11"},name = "Zona estudio")
Map

Map(center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['position', 'transparent…

## Seleccionar una imagen

In [6]:
# Seleccionar una imagen Landsat8
img_L8 = ee.Image("LANDSAT/LC08/C01/T1_SR/LC08_003069_20190808")\
             .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7'],
                     ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])\
             .multiply(0.0001)\
             .clip(zona_ee)\
             .reproject(crs="EPSG:32719", scale = 30)

In [7]:
# Nombre de las bandas
img_L8.bandNames().getInfo()

['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']

In [8]:
# Simbologia escala 0 - 1
viz_es = {
    'bands': ['SWIR1','NIR','RED'],
    'min': 0.1,
    'max': 0.6,
    'gamma': 1.6
}

In [9]:
Map.addLayer(img_L8, viz_es,name = "Landsat8 RS")
Map

Map(bottom=281405.0, center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['positi…

## Crear función de fraccion y endmembers

In [10]:
def calcFractions(image):
    endmembers = [
        [0.0119, 0.0475, 0.0169, 0.625, 0.2399, 0.0675], #gv#
        [0.1514, 0.1597, 0.1421, 0.3053, 0.7707, 0.1975], #npv#
        [0.1799, 0.2479, 0.31580, 0.5437, 0.7707, 0.6646], #soil#
    ]
    bandNames = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']
    outBandNames = ['gv', 'npv', 'soil']
    # Uminxing data
    fractions = ee.Image(image)\
    .select(bandNames)\
    .unmix(endmembers)\
    .max(0)\
    .multiply(100)\
    .byte()\
    .select([0, 1, 2],outBandNames)
    return (fractions)

In [11]:
# calculate 'gv', 'npv', 'soil'
fraction = calcFractions(img_L8)

## Calcular Shade

In [12]:
# calculate SHADE
suma = fraction.select(['gv', 'npv', 'soil']).reduce(ee.Reducer.sum())
shade = suma.subtract(100).abs().byte().rename('shade')

## Calcular GVshade

In [13]:
# calculate GVshade
GVshade = fraction.select(['gv']) \
                      .divide(suma) \
                      .multiply(100) \
                      .byte() \
                      .rename('GVshade')

## Concatenar imagenes

In [14]:
image_frac = fraction.addBands(shade).addBands(GVshade)

## Funcion calcular NDFI

In [15]:
# Calculate NDFI
def Calc_NDFI(image):
  ndfi = image.expression('(GVshade-(npv+soil))/(GVshade+npv+soil)', {
    'GVshade':image.select('GVshade'),
    'npv':image.select('npv'),
    'soil':image.select('soil')
      })
  return ndfi.rename('NDFI')

In [16]:
NDFI = Calc_NDFI(image_frac)

In [17]:
NDFI.bandNames().getInfo()

['NDFI']

In [18]:
# Simbologia NDFI
viz_ndfi = {
    'min': -1,
    'max': 1,
    'palette': ["ffffff","fffcff","fff9ff","fff7ff","fff4ff",
                "fff2ff","ffefff","ffecff","ffeaff","ffe7ff","ffe5ff","ffe2ff",
                "ffe0ff","ffddff","ffdaff","ffd8ff","ffd5ff","ffd3ff","ffd0ff",
                "ffceff","ffcbff","ffc8ff","ffc6ff","ffc3ff","ffc1ff","ffbeff",
                "ffbcff","ffb9ff","ffb6ff","ffb4ff","ffb1ff","ffafff","ffacff",
                "ffaaff","ffa7ff","ffa4ff","ffa2ff","ff9fff","ff9dff","ff9aff",
                "ff97ff","ff95ff","ff92ff","ff90ff","ff8dff","ff8bff","ff88ff",
                "ff85ff","ff83ff","ff80ff","ff7eff","ff7bff","ff79ff","ff76ff",
                "ff73ff","ff71ff","ff6eff","ff6cff","ff69ff","ff67ff","ff64ff",
                "ff61ff","ff5fff","ff5cff","ff5aff","ff57ff","ff55ff","ff52ff",
                "ff4fff","ff4dff","ff4aff","ff48ff","ff45ff","ff42ff","ff40ff",
                "ff3dff","ff3bff","ff38ff","ff36ff","ff33ff","ff30ff","ff2eff",
                "ff2bff","ff29ff","ff26ff","ff24ff","ff21ff","ff1eff","ff1cff",
                "ff19ff","ff17ff","ff14ff","ff12ff","ff0fff","ff0cff","ff0aff",
                "ff07ff","ff05ff","ff02ff","ff00ff","ff00ff","ff0af4","ff15e9",
                "ff1fdf","ff2ad4","ff35c9","ff3fbf","ff4ab4","ff55aa","ff5f9f",
                "ff6a94","ff748a","ff7f7f","ff8a74","ff946a","ff9f5f","ffaa55",
                "ffb44a","ffbf3f","ffc935","ffd42a","ffdf1f","ffe915","fff40a",
                "ffff00","ffff00","fffb00","fff700","fff300","fff000","ffec00",
                "ffe800","ffe400","ffe100","ffdd00","ffd900","ffd500","ffd200",
                "ffce00","ffca00","ffc600","ffc300","ffbf00","ffbb00","ffb700",
                "ffb400","ffb000","ffac00","ffa800","ffa500","ffa500","f7a400",
                "f0a300","e8a200","e1a200","d9a100","d2a000","ca9f00","c39f00",
                "bb9e00","b49d00","ac9c00","a59c00","9d9b00","969a00","8e9900",
                "879900","7f9800","789700","709700","699600","619500","5a9400",
                "529400","4b9300","439200","349100","2d9000","258f00","1e8e00",
                "168e00","0f8d00","078c00","008c00","008c00","008700","008300",
                "007f00","007a00","007600","007200","006e00","006900","006500",
                "006100","005c00","005800","005400","005000","004c00"]
}

In [19]:
Map.addLayer(NDFI, viz_ndfi,name = "L8 NDFI")
Map

Map(bottom=281405.0, center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['positi…

## Exportar Imagen

In [20]:
import os
os.getcwd()

'C:\\Users\\GEOMATICA\\Teledeteccion_Python\\GEE'

In [21]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [22]:
# Exportar imagen directorio
geemap.ee_export_image(NDFI, filename="L8_NDFI.tif", scale=30, region=geometria, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\L8_NDFI.tif


In [23]:
# Exportar en google drive
geemap.ee_export_image_to_drive(NDFI, description='NDFI_L8', folder='GEE_Python', region=geometria, scale=30)

Exporting NDFI_L8 ...
